<font color="#5642C2" size="5">Excersise 2 in Natural Language Processing</font>
<br>
<font color="#5642C2" size="4">Tweet Classification and Sentiment Analysis</font>
<br>
<font color="#5642C2" size="3">Author:</font><font color="#ADADAD" size="3">Angeliki Mylonaki</font>
<br>


In [1]:
import csv
import glob
import os

def getData(regex):
    trainFiles = glob.glob(os.path.join(".", regex)) #make list of paths

    tsvreader=[]

    for fd in trainFiles:
        with open(fd) as tsvfile:
            tsvreader.extend(csv.reader(tsvfile, delimiter="\t"))

    return [line[2] for line in tsvreader], [line[1] for line in tsvreader]
            
def peekOnData(x,y):
    print "Previewing data"
    print "....................................."
    for i in range(1,10):
        print ">> "+ y[i] +": "+x[i]+"\n"
    


<font color="#5642C2" size="5">Tweet Classifier</font>

<font color="#cc00ff" size="3">Reading +peeking on all training data</font>

In [4]:
trainX,trainY=getData("*train*.tsv")
devX,devY=getData("*dev*.tsv")
testX,testY=getData("*test*.tsv")

peekOnData(testX[:10],testY[:10])

Previewing data
.....................................
>> neutral: @fakethom Have android tab and don't use phone much, in fact very little. May go the Sony route then:-)

>> positive: Finally I get my ps4 back I sent it to Sony cause my HDMI was mess up now I can play MG's Tuesday yeaaaaa buddy

>> neutral: Sony's 1st teaser package for the launch of the original Playstation seems to feature a dominatrix? https://t.co/xbisCRkPL4 #MistressSophia

>> neutral: #tv Ind vs SL 3rd Test Day 3: Cricket live score and Sony Six live streaming info: Watch the live teleca... http://t.co/mUlHw4cN00 #Sony

>> neutral: @TruthInsider @bertymufc @gamerxone720 @PNF4LYFE @Yanks2013 @VirtuaMe Lol it's all about Sony Sony Sony, if Sony gave Bj's u be the 1st

>> positive: When you remember Sony is trying to make bible study mandatory on Sunday nights @xo_taylorbang http://t.co/CfljDMvMv3

>> positive: @InfinityInq Everyone is playing 3.0 and I'm just sitting here playing 2.0 because Sony is making me wait 

<font color="#cc00ff" size="3">Mining Tweets</font>

In [5]:
from nltk import corpus
from nltk import word_tokenize
from nltk import sent_tokenize
import string
import re
from autocorrect import spell


def reduce_lengthening(text):
    pattern = re.compile(r"(.)\1{2,}")
    return pattern.sub(r"\1\1", text)

def mineTweets(tweetList):
    minedList=[]
    
    stopWords = corpus.stopwords.words("english")
    tweetList=[s.translate(None, string.punctuation) for s in tweetList]
    
    for tweet in tweetList:
        
        #splitting tweet into phrases
        sentences = sent_tokenize(tweet)
        for sentence in sentences:
            #removing digits
            sentence=sentence.translate(None, string.digits)
            #removing urls
            sentence=re.sub(r'^https?:\/\/.*[\r\n]*', '', sentence, flags=re.MULTILINE)
            #getting words of tweet phrase
            words=word_tokenize(sentence)
            
            #fixing spelling 
            words=[reduce_lengthening(word) for word in words]
            #should uncomment that, but it introduces HUGE DELAY
            #words=[spell(word) for word in words]

            
            #removing stopwords
            sentence = ' '.join([word for word in words
                             if word not in stopWords])
            
            minedList.append(sentence)
            
    return minedList

trainX=mineTweets(trainX)
trainX=trainX[:100]
trainY=trainY[:100]
testX=mineTweets(testX)
testX=testX[:100]
testY=testY[:100]
devX=mineTweets(devX)
devX=devX[:100]
devY=devY[:100]

<font color="#cc00ff" size="3">Vectorizing</font>

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer(stop_words="english", binary="true")
X_train_counts = count_vect.fit_transform(trainX)
X_dev_counts = count_vect.transform(devX)
X_test_counts=  count_vect.transform(testX)

<font color="#cc00ff" size="3"> Calculating TF-IDF</font>

In [7]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
X_dev_tfidf = tfidf_transformer.transform(X_dev_counts)
X_test_tfidf = tfidf_transformer.transform(X_test_counts)

<font color="#cc00ff" size="3"> Training Multiple Classifiers</font>

In [8]:
import numpy as np
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestCentroid
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score



classifiers=[[SGDClassifier(),"Linear Gradient Descent"],
             [RandomForestClassifier(),"Random Forests"],
             [MultinomialNB(),"Multinomial Naive Bayes"],
             [KNeighborsClassifier(),"KNN"],
             [PassiveAggressiveClassifier(),"PassiveAggressive"],
             [LinearSVC(),"LinearSVC"]]

for clf in classifiers:
    text_clf = Pipeline([('vect', CountVectorizer()),
                      ('tfidf', TfidfTransformer()),
                      ('clf', clf[0]),
    ])
    text_clf.fit(trainX, trainY)  
    predicted = text_clf.predict(testX)
    print str(clf[1]+": ").ljust(30),text_clf.score(testX, testY)


/home/gelou/anaconda2/lib/python2.7/site-packages/sklearn/linear_model/stochastic_gradient.py:84: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/home/gelou/anaconda2/lib/python2.7/site-packages/sklearn/linear_model/stochastic_gradient.py:84: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.passive_aggressive.PassiveAggressiveClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


Linear Gradient Descent:       0.44
Random Forests:                0.47
Multinomial Naive Bayes:       0.48
KNN:                           0.35
PassiveAggressive:             0.41
LinearSVC:                     0.44


<font color="#cc00ff" size="3"> Fine Tuning LinearSVC</font>

In [9]:
#tuning hyperParameters
from sklearn.model_selection import RandomizedSearchCV
import scipy 

#Using RandomizedSearch to tube SVN hyoeroaraneter C

param_grid = {'C': scipy.stats.expon(scale=100), 
              'dual': [True,False],
              'tol': scipy.stats.expon(scale=.1)}

rsearch = RandomizedSearchCV(estimator=LinearSVC(), param_distributions=param_grid, n_iter=100, random_state=7)
rsearch.fit(X_dev_tfidf,devY)

predicted = rsearch.predict(X_test_tfidf)
print str("LinearSVC tuned: ").ljust(30),np.mean(predicted == testY)

LinearSVC tuned:               0.84


<font color="#5642C2" size="5">Introducing Sentiment Analysis</font>

<font color="#cc00ff" size="3">Function to calculate polarity for all Tweets</font>

In [16]:
def calculatePolarity(tweets):
    positive=0
    negative=0
    neutral=0
    predicted=[]
    
    for tweet in tweets:
        print tweet.classify()
        if tweet.sentiment.polarity>0:
            positive+=1
            predicted.append("positive")
            print "positive\n"
        elif tweet.sentiment.polarity<0:
            negative+=1
            predicted.append("negative")
            print "negative\n"
        else:
            neutral+=1
            predicted.append("neutral")
            print "neutral\n"
    print "Negative tweets percentage: ".ljust(30),100*float(negative)/len(tweets),"%"
    print "Neutral tweets percentage: ".ljust(30), 100*float(neutral)/len(tweets),"%"
    print "Positive tweets percentage: ".ljust(30),100*float(positive)/len(tweets),"%"


    return predicted


<font color="#cc00ff" size="3">A bit more mining</font>

In [ ]:
from textblob import TextBlob
from textblob.classifiers import NaiveBayesClassifier
from textblob.classifiers import DecisionTreeClassifier
from textblob.classifiers import MaxEntClassifier



def mineTweetsTextBlob(cl,tweets):
    objList=[]
    for tweet in tweets:
        twt=TextBlob(tweet,classifier=cl)
        #convertin to lower case
        twt=twt.lower()

        #attempting to correct spelling mistakes
        twt=twt.correct()

        temp=""
        for words in twt.words:
            temp+=words.lemmatize()+" "
        
        objList.append(TextBlob(temp,classifier=cl))
        #objList.append(twt)
    return objList

def trainClassifier(data,label):
    tplList=[]
    for i,j in zip(trainX,trainY):
        tplList.append((i,j))
    
    return NaiveBayesClassifier(tplList)
    #clDT=DecisionTreeClassifier(tplList)
    #clMaxEnt=MaxEntClassifier(tplList)

    
cl=trainClassifier(trainX,trainY)
textBlobs=mineTweetsTextBlob(cl,testX)
predicted=calculatePolarity(textBlobs)

equal=0
not_equal=0
for i,j in zip(predicted,trainY):
    
    if i==j:
        equal+=1
    else:
        not_equal+=1
print equal,not_equal



print str("\n\nTextBlob accuracy:").ljust(30),